In [14]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma
from pyalgotrade import plotter
from pyalgotrade import broker


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        strategy.BacktestingStrategy.__init__(self, feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))
        self.__position.exitStop(execInfo.getPrice()*0.95,True)

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        
        if position.getExitOrder().getType() == broker.Order.Type.STOP:
            order_type = "stop"
        else:
            order_type = "market"
            
        self.info("SELL at $%.2f with a %s order" % (execInfo.getPrice(),order_type))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        #if self.__sma[-1] is None:
          #  return

        #bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            
            #if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
            self.__position = self.enterLong(self.__instrument, 10, True)
        elif self.__position.getAge().days > 1:
            self.__position.exitMarket()
        # Check if we have to exit the position.
        #elif bar.getPrice() < self.__sma[-1]:
            #self.__position.exitMarket()


def run_strategy(smaPeriod):
   
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV("data","quandl.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "data", smaPeriod)
    plt = plotter.StrategyPlotter(myStrategy)
    
    
    myStrategy.run()
    plt.plot()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(15)

2000-01-04 00:00:00 strategy [INFO] BUY at $26.09
2000-01-05 00:00:00 strategy [INFO] SELL at $23.13 with a stop order
2000-01-06 00:00:00 strategy [INFO] BUY at $22.64
2000-01-07 00:00:00 strategy [INFO] SELL at $21.46 with a stop order
2000-01-10 00:00:00 strategy [INFO] BUY at $24.40


Exception: Exit order is active and it should be canceled first